In [1]:
# https://deeplearning4j.org/model-import-keras
# https://github.com/amir-abdi/Deep-Learning-Papers-Reading-Roadmap
# http://www.deeplearningbook.org/

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import tensorflow as tf

from sklearn.model_selection import train_test_split #pip install -U scikit-learn
from sklearn.preprocessing import MinMaxScaler

from keras import models
from keras import layers
from keras import optimizers
from keras import losses
from keras import metrics


Using TensorFlow backend.


In [2]:
#pip install --upgrade keras
#pip install --upgrade tensorflow
#...

In [3]:
#Loading the data
datos = pd.read_csv('datos.csv')

#Shuffle
#np.random.shuffle(datos)
#datos = datos.sample(frac=1).reset_index(drop=True)
datos.sample(frac=1)

x_data = datos.drop(labels=['PTO_X', 'PTO_Y'], axis=1)
y_data = datos.drop(['WI0', 'WI1', 'WI2', 'WI3', 'WI4', 'WI5', 'WI6', 'WI7', 'WI8', 'WI9'], axis=1)
x_train, x_test, y_train, y_test = train_test_split(x_data, y_data, test_size=0.3, random_state=101)

print("TRAIN DATA: ", x_train.shape)
print("TEST  DATA: ", x_test.shape)

TRAIN DATA:  (1539, 10)
TEST  DATA:  (660, 10)


In [4]:
#Normalizing the data

"""
scaler = MinMaxScaler()
scaler.fit(x_train)
scaler.fit(x_test)
"""
#OR
"""
mean = x_train.mean(axis=0)
x_train -= mean
std = x_train.std(axis=0)
x_train /= std
x_test -= mean
x_test /= std
#--> Despreciable, si no lo haces en x_test deberías ver que no cambia error
print("MEAN:\n",mean, "\nSTD DEV:\n",std)
"""
#OR
"""
min = x_train.min().min()
max = x_train.max().max()
print("MIN: %f  MAX: %f" % (min, max))
x_train = (x_train - min)/(max - min)
x_test = (x_test - min)/(max - min)
"""
#OR
mean = x_train.mean(axis=0).mean()
std  = x_train.std(axis=0).std()
print("MEAN: %f,  STD DEV: %f" % (mean, std))
x_train = (x_train - mean) / std
x_test = (x_test - mean) / std


In [ ]:
#separar validate & test data ?
x_valid = x_train[:200]
x_train2 = x_train[200:]
y_valid = y_train[:200]
y_train2 = y_train[200:]

In [6]:
#ANN Model
def build_model():
    model = models.Sequential()
    model.add(layers.Dense(7, activation='relu', input_shape=(x_data.shape[1],), name='cesin'))
    model.add(layers.Dense(7, activation='relu'))
    model.add(layers.Dense(7, activation='relu'))
    model.add(layers.Dense(2, name='cesout'))
    model.compile(optimizer=optimizers.RMSprop(lr=0.00005), loss=losses.mse, metrics=[metrics.mae])
    #model.compile(optimizer='rmsprop', loss='mse', metrics=['mae'])
    return model

In [7]:
#Train the model  (ep: 22000)
model = build_model()
#history = model.fit(x_train, y_train, validation_data=(x_test, y_test), batch_size=100, verbose=0)
hist = model.fit(x_train, y_train, validation_data=(x_test, y_test), epochs=22000, batch_size=100, verbose=0)
#history = model.fit(x_train2, y_train2, validation_data=(x_valid, y_valid), epochs=15000, batch_size=100, verbose=0)

In [9]:
#Study the model

print("Min error %f at epoch %d" % (np.min(hist.history['val_loss'][:]), np.argmin(hist.history['val_loss'][:])))

loss = hist.history['loss'][2000:]
val_loss = hist.history['val_loss'][2000:]

epochs = range(1, len(loss) + 1)
plt.plot(epochs, loss, 'bo', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

NameError: name 'hist' is not defined

In [ ]:
test_mse_score, test_mae_score = model.evaluate(x_test, y_test)
test_mae_score

In [10]:
#CES error
y_predicted = model.predict(x_test)
y_test0 = np.asarray(y_test)
print("-------", type(y_test), " -- ", type(y_test0))
print(y_predicted.shape)
print("\n")

#
def calcError(real, pred):
    error = 0.
    real0 = np.asarray(real)
    for i in range(real.shape[0]):
        error = error + np.sqrt(pow(pred[i][0] - real0[i][0], 2) + pow(pred[i][1] - real0[i][1], 2))
    return error  /  real.shape[0]
print("CES ERROR: ", calcError(y_test0, y_predicted))
print("\n")

#
for i in range(1,10):
    print("---------------> Real vs Pred.: (%.0f, %.0f) vs (%.0f, %.0f)"
          % (y_test0[i][0], y_test0[i][1], y_predicted[i][0], y_predicted[i][1]))
print("\n")

------- <class 'pandas.core.frame.DataFrame'>  --  <class 'numpy.ndarray'>
(660, 2)


CES ERROR:  17.329334148597265


---------------> Real vs Pred.: (69, 29) vs (54, 40)
---------------> Real vs Pred.: (69, 31) vs (57, 43)
---------------> Real vs Pred.: (77, 87) vs (83, 73)
---------------> Real vs Pred.: (38, 87) vs (70, 58)
---------------> Real vs Pred.: (85, 77) vs (88, 79)
---------------> Real vs Pred.: (73, 51) vs (66, 53)
---------------> Real vs Pred.: (65, 55) vs (62, 48)
---------------> Real vs Pred.: (46, 84) vs (77, 65)
---------------> Real vs Pred.: (78, 68) vs (84, 74)




In [ ]:
#################################################################################

In [ ]:
#Save the Keras model
model.save('cesnet.h5')# creates a HDF5 file 'my_model.h5'
#del model  # deletes the existing model
from keras.models import load_model
model2 = load_model('cesnet.h5')
model3 = load_model('cesnet - error7.6.h5')

#
y_predicted = model2.predict(x_test)
del model2
print("CES ERROR: ", calcError(y_test, y_predicted))
#
y_predicted = model3.predict(x_test)
del model3
print("CES ERROR: ", calcError(y_test, y_predicted))

In [ ]:
#Save the Protobuf (Tensorflow) model
#python keras_to_tf.py -input_model_file cesnet.h5 -output_model_file cesnet.pb
%run keras_to_tf.py -input_model_file cesnet.h5 -output_model_file cesnet.pb

In [ ]:
#################################################################################

In [ ]:
#Load the Protobuf (Tensorflow) model
#https://stackoverflow.com/questions/44274701/make-predictions-using-a-tensorflow-graph-from-a-keras-model
def load_graph(frozen_graph_filename):
    # We load the protobuf file from the disk and parse it to retrieve the unserialized graph_def
    with tf.gfile.GFile(frozen_graph_filename, "rb") as f:
        graph_def = tf.GraphDef()
        graph_def.ParseFromString(f.read())

    # Then, we can use again a convenient built-in function to import a graph_def into the current default Graph
    with tf.Graph().as_default() as graph:
        tf.import_graph_def(
            graph_def, 
            input_map=None, 
            return_elements=None, 
            name="prefix", 
            op_dict=None, 
            producer_op_list=None
        )

    input_name = graph.get_operations()[0].name+':0'     #prefix/dense_1_input:0
    output_name = graph.get_operations()[-1].name+':0'   #prefix/output_node0:0

    return graph, input_name, output_name

#g, i, o = load_graph('cesnet.pb')

In [ ]:
#Run the Protobuf (Tensorflow) model
def predict(model_path, input_data):
    # load tf graph
    tf_model, tf_input, tf_output = load_graph(model_path)

    # Create tensors for model input and output
    x = tf_model.get_tensor_by_name(tf_input)
    y = tf_model.get_tensor_by_name(tf_output) 

    # Number of model outputs
    num_outputs = y.shape.as_list()[1]
    print(y.shape)
    print(num_outputs)
    print(input_data.shape[0])
    predictions = np.zeros((input_data.shape[0], num_outputs))
    for i in range(input_data.shape[0]):        
        with tf.Session(graph=tf_model) as sess:
            y_out = sess.run(y, feed_dict={x: input_data[i:i+1]})
            predictions[i] = y_out

    return predictions

In [ ]:
tf_predictions = predict('cesnet.pb', x_test)

In [ ]:
print("CES ERROR: ", calcError(y_test, tf_predictions))
print("\n")

TODO: Importar en Android
https://deeplearning4j.org/model-import-keras

HDF5 isn't supported on Android, so import on the desktop, save, and load on Android.
net = KerasModelImport.importKerasSequentialModelAndWeights( dir + "cesnet.h5" )

Problem arises if use Keras-2. So, tried to save model using Keras-1.1.0 and it works perfectly.
Also able to import the model on android.

$ pip install keras==1.2
$ python -c "import keras; print(keras.__version__)"

Tampoco vale pasar a keras 1, porque cambiaría la estructura de mi red,
mejor esperar a DL4J 0.92 que si permite la importación desde keras 2
(o compilar el actual snapshot que tambien soporta keras 2)

In [ ]:
import keras; print(keras.__version__)